In [0]:
%pip install --quiet airspace

In [0]:
dbutils.library.restartPython()

In [0]:
# Setup
import datetime
import logging
import os
import sys

import numpy as np
import pandas as pd

import airspace

from effodata import ACDS, golden_rules, Sifter, Equality, Joiner, Lambda
from kpi_metrics import KPI, available_metrics as  amKPI, get_metrics, AliasMetric, AliasGroupby, CustomMetric

import seg

from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    FloatType,
    DateType,
    TimestampType,
)
from pyspark.sql.window import Window
from pyspark.sql.functions import to_date, regexp_replace
from delta.tables import DeltaTable
from kayday import KrogerDate
import dateutil.relativedelta
# spark = SparkSession.builder.getOrCreate()
logging.getLogger().setLevel(logging.INFO)


In [0]:
config = {
  'start_date': '20240802',
  'end_date' : '20250130'
}

# Configure FUNLO, ACDs, ETC

In [0]:
funlo = seg.get_seg_for_date(
  'funlo',
'2025-01-30', 
  )

In [0]:
active_guids_ehhns = airspace.email_households(config)

In [0]:
cds = seg.get_seg_for_date('cds_4_hh', '20250130')

In [0]:
acds = ACDS()

In [0]:
transactions = acds.get_transactions(
  start_date='20240802', 
  end_date="20250130")

# The Control Group

##Read in control tables

And Verify that logic for getting entire group Open rate makes sense


In [0]:
control = spark.read.table("sandbox_dev.tm_learning.final_all_control")
control.select("guid").distinct().count()

In [0]:

control = control.join(active_guids_ehhns, on = "guid", how  = "inner")

##Getting  Open Rate for Overall Group

In [0]:
control_sum = control.select("guid", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
control_sum.agg(f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

## Getting Unsubscribe Rates for the overall group

In [0]:
control_sum = control.select("guid", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
control_sum.agg(f.sum("DEPT_UNSUBSCRIBE") / f.sum("DEPT_OPEN")).display()

## OPEN RATE BY OLD TAGS

In [0]:
control_old_tags = control.select("guid", "EMAIL_TYPE_OLD_TAG", "EMAIL_TYPE_OLD_TAG_OPEN","EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE", "EMAIL_TYPE_OLD_TAG_SENT").dropDuplicates()

control_old_tags.groupBy("EMAIL_TYPE_OLD_TAG").agg(f.sum("EMAIL_TYPE_OLD_TAG_OPEN"). f.sum("EMAIL_TYPE_OLD_TAG_SENT"),
                                               f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"), f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).display()

In [0]:

control_old_tags = control.select("guid", "EMAIL_TYPE_OLD_TAG", "EMAIL_TYPE_OLD_TAG_OPEN","EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE", "EMAIL_TYPE_OLD_TAG_SENT").dropDuplicates()

control_old_tags.groupBy("EMAIL_TYPE_OLD_TAG").agg(f.sum("EMAIL_TYPE_OLD_TAG_OPEN"). f.sum("EMAIL_TYPE_OLD_TAG_SENT"),
                                               f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"), f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).display()

## FUNLO X CONTROL

In [0]:
controlo = control.join(funlo, on = "ehhn", how = "inner")

In [0]:
controlo_sum = controlo.select("guid", "funlo_rollup_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
controlo_sum.groupBy("funlo_rollup_desc").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

In [0]:
controlo_sum = controlo.select("guid", "funlo_rollup_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
controlo_sum.groupBy("funlo_rollup_desc").agg(f.sum("DEPT_UNSUBSCRIBE"), f.sum("DEPT_OPEN"), f.sum("DEPT_UNSUBSCRIBE") / f.sum("DEPT_OPEN")).display()

In [0]:
controlo_sum = controlo.select("guid", "funlo_seg_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
controlo_sum.groupBy("funlo_seg_desc").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

In [0]:
controlo_sum = controlo.select("guid", "funlo_seg_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
controlo_sum.groupBy("funlo_seg_desc").agg(f.sum("DEPT_UNSUBSCRIBE"), f.sum("DEPT_OPEN"), f.sum("DEPT_UNSUBSCRIBE") / f.sum("DEPT_OPEN")).display()

##FUNLO X CONTROL X OLD TAGS

In [0]:
controlo.columns

In [0]:
funlo_old_aggs_rollup = controlo.groupBy("funlo_rollup_desc", "EMAIL_TYPE_OLD_TAG", "CATEGORY1_OLD_TAG").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("EMAIL_TYPE_OLD_TAG_SENT").alias("EMAIL_TYPE_OLD_TAG_SENT"),
  f.sum("EMAIL_TYPE_OLD_TAG_OPEN").alias("EMAIL_TYPE_OLD_TAG_OPEN"),
  f.sum("EMAIL_TYPE_OLD_TAG_CLICK").alias("EMAIL_TYPE_OLD_TAG_CLICK"),
  f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE").alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_OPEN_RATE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_CLICK") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE_RATE"),
  f.sum("CATEGORY1_OLD_TAG_SENT").alias("CATEGORY1_OLD_TAG_SENT"),
  f.sum("CATEGORY1_OLD_TAG_OPEN").alias("CATEGORY1_OLD_TAG_OPEN"),
  f.sum("CATEGORY1_OLD_TAG_CLICK").alias("CATEGORY1_OLD_TAG_CLICK"),
  f.sum("CATEGORY1_OLD_TAG_UNSUBSCRIBE").alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("CATEGORY1_OLD_TAG_OPEN") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_OPEN_RATE"),
  (f.sum("CATEGORY1_OLD_TAG_CLICK") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE_RATE"),
)


In [0]:
funlo_old_aggs_rollup.display()

In [0]:
funlo_old_aggs_seg = controlo.groupBy("funlo_seg_desc", "EMAIL_TYPE_OLD_TAG", "CATEGORY1_OLD_TAG").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("EMAIL_TYPE_OLD_TAG_SENT").alias("EMAIL_TYPE_OLD_TAG_SENT"),
  f.sum("EMAIL_TYPE_OLD_TAG_OPEN").alias("EMAIL_TYPE_OLD_TAG_OPEN"),
  f.sum("EMAIL_TYPE_OLD_TAG_CLICK").alias("EMAIL_TYPE_OLD_TAG_CLICK"),
  f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE").alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_OPEN_RATE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_CLICK") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE_RATE"),
  f.sum("CATEGORY1_OLD_TAG_SENT").alias("CATEGORY1_OLD_TAG_SENT"),
  f.sum("CATEGORY1_OLD_TAG_OPEN").alias("CATEGORY1_OLD_TAG_OPEN"),
  f.sum("CATEGORY1_OLD_TAG_CLICK").alias("CATEGORY1_OLD_TAG_CLICK"),
  f.sum("CATEGORY1_OLD_TAG_UNSUBSCRIBE").alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("CATEGORY1_OLD_TAG_OPEN") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_OPEN_RATE"),
  (f.sum("CATEGORY1_OLD_TAG_CLICK") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE_RATE"),
)

In [0]:
funlo_old_aggs_seg.display()

##CDS X CONTROL

In [0]:
control_cds = control.join(cds, on = "ehhn", how = "inner")

###Price Dim Seg

In [0]:
price_dim_aggs = control_cds.groupBy("price_dim_seg", "EMAIL_TYPE_OLD_TAG", "CATEGORY1_OLD_TAG").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("EMAIL_TYPE_OLD_TAG_SENT").alias("EMAIL_TYPE_OLD_TAG_SENT"),
  f.sum("EMAIL_TYPE_OLD_TAG_OPEN").alias("EMAIL_TYPE_OLD_TAG_OPEN"),
  f.sum("EMAIL_TYPE_OLD_TAG_CLICK").alias("EMAIL_TYPE_OLD_TAG_CLICK"),
  f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE").alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_OPEN_RATE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_CLICK") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE_RATE"),
  f.sum("CATEGORY1_OLD_TAG_SENT").alias("CATEGORY1_OLD_TAG_SENT"),
  f.sum("CATEGORY1_OLD_TAG_OPEN").alias("CATEGORY1_OLD_TAG_OPEN"),
  f.sum("CATEGORY1_OLD_TAG_CLICK").alias("CATEGORY1_OLD_TAG_CLICK"),
  f.sum("CATEGORY1_OLD_TAG_UNSUBSCRIBE").alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("CATEGORY1_OLD_TAG_OPEN") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_OPEN_RATE"),
  (f.sum("CATEGORY1_OLD_TAG_CLICK") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE_RATE"),
)

In [0]:
price_dim_aggs.display()

In [0]:
price_dim_aggs_solo = price_dim_aggs.select("price_dim_seg", "EMAIL_TYPE_OLD_TAG", "EMAIL_TYPE_OLD_TAG_OPEN","EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE", "EMAIL_TYPE_OLD_TAG_SENT").dropDuplicates()

price_dim_aggs_solo.groupBy("price_dim_seg").agg(f.sum("EMAIL_TYPE_OLD_TAG_OPEN"), f.sum("EMAIL_TYPE_OLD_TAG_SENT"), f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).display()

### Health Dim Seg

In [0]:
health_dim_aggs = control_cds.groupBy("health_dim_seg", "EMAIL_TYPE_OLD_TAG", "CATEGORY1_OLD_TAG").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("EMAIL_TYPE_OLD_TAG_SENT").alias("EMAIL_TYPE_OLD_TAG_SENT"),
  f.sum("EMAIL_TYPE_OLD_TAG_OPEN").alias("EMAIL_TYPE_OLD_TAG_OPEN"),
  f.sum("EMAIL_TYPE_OLD_TAG_CLICK").alias("EMAIL_TYPE_OLD_TAG_CLICK"),
  f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE").alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_OPEN_RATE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_CLICK") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE_RATE"),
  f.sum("CATEGORY1_OLD_TAG_SENT").alias("CATEGORY1_OLD_TAG_SENT"),
  f.sum("CATEGORY1_OLD_TAG_OPEN").alias("CATEGORY1_OLD_TAG_OPEN"),
  f.sum("CATEGORY1_OLD_TAG_CLICK").alias("CATEGORY1_OLD_TAG_CLICK"),
  f.sum("CATEGORY1_OLD_TAG_UNSUBSCRIBE").alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("CATEGORY1_OLD_TAG_OPEN") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_OPEN_RATE"),
  (f.sum("CATEGORY1_OLD_TAG_CLICK") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE_RATE"),
)

In [0]:
health_dim_aggs.display()

In [0]:
health_dim_aggs_solo = health_dim_aggs.select("health_dim_seg", "EMAIL_TYPE_OLD_TAG", "EMAIL_TYPE_OLD_TAG_OPEN","EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE", "EMAIL_TYPE_OLD_TAG_SENT").dropDuplicates()

health_dim_aggs_solo.groupBy("health_dim_seg").agg(f.sum("EMAIL_TYPE_OLD_TAG_OPEN"), f.sum("EMAIL_TYPE_OLD_TAG_SENT"), f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).display()

### Qualtity Dim Seg

In [0]:
quality_dim_aggs = control_cds.groupBy("quality_dim_seg", "EMAIL_TYPE_OLD_TAG", "CATEGORY1_OLD_TAG").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("EMAIL_TYPE_OLD_TAG_SENT").alias("EMAIL_TYPE_OLD_TAG_SENT"),
  f.sum("EMAIL_TYPE_OLD_TAG_OPEN").alias("EMAIL_TYPE_OLD_TAG_OPEN"),
  f.sum("EMAIL_TYPE_OLD_TAG_CLICK").alias("EMAIL_TYPE_OLD_TAG_CLICK"),
  f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE").alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_OPEN_RATE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_CLICK") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE_RATE"),
  f.sum("CATEGORY1_OLD_TAG_SENT").alias("CATEGORY1_OLD_TAG_SENT"),
  f.sum("CATEGORY1_OLD_TAG_OPEN").alias("CATEGORY1_OLD_TAG_OPEN"),
  f.sum("CATEGORY1_OLD_TAG_CLICK").alias("CATEGORY1_OLD_TAG_CLICK"),
  f.sum("CATEGORY1_OLD_TAG_UNSUBSCRIBE").alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("CATEGORY1_OLD_TAG_OPEN") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_OPEN_RATE"),
  (f.sum("CATEGORY1_OLD_TAG_CLICK") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE_RATE"),
)

In [0]:
quality_dim_aggs.display()

In [0]:
quality_dim_aggs_solo = quality_dim_aggs.select("quality_dim_seg", "EMAIL_TYPE_OLD_TAG", "EMAIL_TYPE_OLD_TAG_OPEN","EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE", "EMAIL_TYPE_OLD_TAG_SENT").dropDuplicates()

quality_dim_aggs_solo.groupBy("quality_dim_seg").agg(f.sum("EMAIL_TYPE_OLD_TAG_OPEN"), f.sum("EMAIL_TYPE_OLD_TAG_SENT"), f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).display()

### Convenience dim seg

In [0]:
convenience_dim_aggs = control_cds.groupBy("convenience_dim_seg", "EMAIL_TYPE_OLD_TAG", "CATEGORY1_OLD_TAG").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("EMAIL_TYPE_OLD_TAG_SENT").alias("EMAIL_TYPE_OLD_TAG_SENT"),
  f.sum("EMAIL_TYPE_OLD_TAG_OPEN").alias("EMAIL_TYPE_OLD_TAG_OPEN"),
  f.sum("EMAIL_TYPE_OLD_TAG_CLICK").alias("EMAIL_TYPE_OLD_TAG_CLICK"),
  f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE").alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_OPEN_RATE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_CLICK") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE_RATE"),
  f.sum("CATEGORY1_OLD_TAG_SENT").alias("CATEGORY1_OLD_TAG_SENT"),
  f.sum("CATEGORY1_OLD_TAG_OPEN").alias("CATEGORY1_OLD_TAG_OPEN"),
  f.sum("CATEGORY1_OLD_TAG_CLICK").alias("CATEGORY1_OLD_TAG_CLICK"),
  f.sum("CATEGORY1_OLD_TAG_UNSUBSCRIBE").alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("CATEGORY1_OLD_TAG_OPEN") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_OPEN_RATE"),
  (f.sum("CATEGORY1_OLD_TAG_CLICK") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE_RATE"),
)

In [0]:
convenience_dim_aggs.display()

In [0]:
convenience_dim_aggs_solo = convenience_dim_aggs.select("convenience_dim_seg", "EMAIL_TYPE_OLD_TAG", "EMAIL_TYPE_OLD_TAG_OPEN","EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE", "EMAIL_TYPE_OLD_TAG_SENT").dropDuplicates()


convenience_dim_aggs_solo.groupBy("convenience_dim_seg").agg(f.sum("EMAIL_TYPE_OLD_TAG_OPEN"), f.sum("EMAIL_TYPE_OLD_TAG_SENT"), f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).display()

### Variety Seeking Dim Seg

In [0]:
variety_seeking_dim_aggs = control_cds.groupBy("variety_seeking_dim_seg", "EMAIL_TYPE_OLD_TAG", "CATEGORY1_OLD_TAG").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("EMAIL_TYPE_OLD_TAG_SENT").alias("EMAIL_TYPE_OLD_TAG_SENT"),
  f.sum("EMAIL_TYPE_OLD_TAG_OPEN").alias("EMAIL_TYPE_OLD_TAG_OPEN"),
  f.sum("EMAIL_TYPE_OLD_TAG_CLICK").alias("EMAIL_TYPE_OLD_TAG_CLICK"),
  f.sum("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE").alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_OPEN_RATE"),
  (f.sum("EMAIL_TYPE_OLD_TAG_CLICK") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).alias("EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE_RATE"),
  f.sum("CATEGORY1_OLD_TAG_SENT").alias("CATEGORY1_OLD_TAG_SENT"),
  f.sum("CATEGORY1_OLD_TAG_OPEN").alias("CATEGORY1_OLD_TAG_OPEN"),
  f.sum("CATEGORY1_OLD_TAG_CLICK").alias("CATEGORY1_OLD_TAG_CLICK"),
  f.sum("CATEGORY1_OLD_TAG_UNSUBSCRIBE").alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE"),
  (f.sum("CATEGORY1_OLD_TAG_OPEN") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_OPEN_RATE"),
  (f.sum("CATEGORY1_OLD_TAG_CLICK") / f.sum("CATEGORY1_OLD_TAG_SENT")).alias("CATEGORY1_OLD_TAG_UNSUBSCRIBE_RATE"),
)

In [0]:
variety_seeking_dim_aggs.display()

In [0]:
variety_seeking_dim_aggs_solo = variety_seeking_dim_aggs.select("variety_seeking_dim_seg", "EMAIL_TYPE_OLD_TAG", "EMAIL_TYPE_OLD_TAG_OPEN","EMAIL_TYPE_OLD_TAG_UNSUBSCRIBE", "EMAIL_TYPE_OLD_TAG_SENT").dropDuplicates()


variety_seeking_dim_aggs_solo.groupBy("variety_seeking_dim_seg").agg(f.sum("EMAIL_TYPE_OLD_TAG_OPEN"), f.sum("EMAIL_TYPE_OLD_TAG_SENT"), f.sum("EMAIL_TYPE_OLD_TAG_OPEN") / f.sum("EMAIL_TYPE_OLD_TAG_SENT")).display()

##ACDS X Control

#The Test Group

## Read in Test Tables

In [0]:
test = spark.read.table("sandbox_dev.tm_learning.final_all_test")
print(test.select("guid").distinct().count())

In [0]:
test = test.join(active_guids_ehhns, on = "guid", how  = "inner")

## Getting open rates for overall group

In [0]:
test_sum = test.select("guid", "DEPT", "DEPT_OPEN", "DEPT_SENT").dropDuplicates()
test_sum.agg(f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

## Getting Unsubscribe Rates for the overall group

In [0]:
test_sum = test.select("guid", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
test_sum.agg(f.sum("DEPT_UNSUBSCRIBE") / f.sum("DEPT_OPEN")).display()

## OPEN RATE BY NEW TAGS

In [0]:
test_old_tags = test.select("guid", "THEME", "THEME_OPEN","THEME_UNSUBSCRIBE", "THEME_SENT").dropDuplicates()

test_old_tags.groupBy("THEME").agg(f.sum("THEME_OPEN"). f.sum("THEME_SENT"),
                                               f.sum("THEME_UNSUBSCRIBE"), f.sum("THEME_OPEN") / f.sum("THEME_SENT")).display()

In [0]:
test_old_tags = test.select("guid", "DEPT", "THEME_OPEN","THEME_UNSUBSCRIBE", "THEME_SENT").dropDuplicates()

test_old_tags.groupBy("DEPT").agg(f.sum("DEPT_OPEN"). f.sum("DEPT_SENT"),
                                               f.sum("DEPT_UNSUBSCRIBE"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

## FUNLO X TEST

In [0]:
testlo = test.join(funlo, on = "ehhn", how = "inner")

In [0]:
testlo_sum = testlo.select("guid", "funlo_rollup_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
testlo_sum.groupBy("funlo_rollup_desc").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

Databricks visualization. Run in Databricks to view.

In [0]:
testlo_sum = testlo.select("guid", "funlo_rollup_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
testlo_sum.groupBy("funlo_rollup_desc").agg(f.sum("DEPT_UNSUBSCRIBE"), f.sum("DEPT_OPEN"), f.sum("DEPT_UNSUBSCRIBE") / f.sum("DEPT_OPEN")).display()

In [0]:
testlo_sum = testlo.select("guid", "funlo_seg_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
testlo_sum.groupBy("funlo_seg_desc").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

Databricks visualization. Run in Databricks to view.

In [0]:
testlo_sum = testlo.select("guid", "funlo_seg_desc", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
testlo_sum.groupBy("funlo_seg_desc").agg(f.sum("DEPT_UNSUBSCRIBE"), f.sum("DEPT_OPEN"), f.sum("DEPT_UNSUBSCRIBE") / f.sum("DEPT_OPEN")).display()

## FUNLO X TEST X NEW_TAGS

In [0]:
funlo_new_aggs_rollup = testlo.groupBy("funlo_rollup_desc", "THEME", "DEPT").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("THEME_SENT").alias("THEME_SENT"),
  f.sum("THEME_OPEN").alias("THEME_OPEN"),
  f.sum("THEME_CLICK").alias("THEME_CLICK"),
  f.sum("THEME_UNSUBSCRIBE").alias("THEME_UNSUBSCRIBE"),
  (f.sum("THEME_OPEN") / f.sum("THEME_SENT")).alias("theme_open_rate"),
  (f.sum("THEME_CLICK") / f.sum("THEME_SENT")).alias("theme_click_rate"),
  f.sum("DEPT_SENT").alias("DEPT_SENT"),
  f.sum("DEPT_OPEN").alias("DEPT_OPEN"),
  f.sum("DEPT_CLICK").alias("DEPT_CLICK"),
  f.sum("DEPT_UNSUBSCRIBE").alias("DEPT_UNSUBSCRIBE"),
  (f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).alias("dept_open_rate"),
  (f.sum("DEPT_CLICK") / f.sum("DEPT_SENT")).alias("dept_click_rate"),
)

In [0]:
funlo_new_aggs_rollup.display()

In [0]:
funlo_new_aggs_seg = testlo.groupBy("funlo_seg_desc", "THEME", "DEPT").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("THEME_SENT").alias("THEME_SENT"),
  f.sum("THEME_OPEN").alias("THEME_OPEN"),
  f.sum("THEME_CLICK").alias("THEME_CLICK"),
  f.sum("THEME_UNSUBSCRIBE").alias("THEME_UNSUBSCRIBE"),
  (f.sum("THEME_OPEN") / f.sum("THEME_SENT")).alias("theme_open_rate"),
  (f.sum("THEME_CLICK") / f.sum("THEME_SENT")).alias("theme_click_rate"),
  f.sum("DEPT_SENT").alias("DEPT_SENT"),
  f.sum("DEPT_OPEN").alias("DEPT_OPEN"),
  f.sum("DEPT_CLICK").alias("DEPT_CLICK"),
  f.sum("DEPT_UNSUBSCRIBE").alias("DEPT_UNSUBSCRIBE"),
  (f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).alias("dept_open_rate"),
  (f.sum("DEPT_CLICK") / f.sum("DEPT_SENT")).alias("dept_click_rate"),
)

In [0]:
funlo_new_aggs_seg.display()

##CDS X TEST

In [0]:
test_cds = test.join(cds, on = "ehhn", how = "inner")

###Price Dim Seg

In [0]:
test_price_aggs = test_cds.groupBy("price_dim_seg", "THEME", "DEPT").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("THEME_SENT").alias("THEME_SENT"),
  f.sum("THEME_OPEN").alias("THEME_OPEN"),
  f.sum("THEME_CLICK").alias("THEME_CLICK"),
  f.sum("THEME_UNSUBSCRIBE").alias("THEME_UNSUBSCRIBE"),
  (f.sum("THEME_OPEN") / f.sum("THEME_SENT")).alias("theme_open_rate"),
  (f.sum("THEME_CLICK") / f.sum("THEME_SENT")).alias("theme_click_rate"),
  f.sum("DEPT_SENT").alias("DEPT_SENT"),
  f.sum("DEPT_OPEN").alias("DEPT_OPEN"),
  f.sum("DEPT_CLICK").alias("DEPT_CLICK"),
  f.sum("DEPT_UNSUBSCRIBE").alias("DEPT_UNSUBSCRIBE"),
  (f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).alias("dept_open_rate"),
  (f.sum("DEPT_CLICK") / f.sum("DEPT_SENT")).alias("dept_click_rate"),
)

In [0]:
test_price_aggs.display()

In [0]:
test_price_aggs_solo = test_price_aggs.select("price_dim_seg", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
test_price_aggs_solo.groupBy("price_dim_seg").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

### Health Dim Seg

In [0]:

test_health_aggs = test_cds.groupBy("health_dim_seg", "THEME", "DEPT").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("THEME_SENT").alias("THEME_SENT"),
  f.sum("THEME_OPEN").alias("THEME_OPEN"),
  f.sum("THEME_CLICK").alias("THEME_CLICK"),
  f.sum("THEME_UNSUBSCRIBE").alias("THEME_UNSUBSCRIBE"),
  (f.sum("THEME_OPEN") / f.sum("THEME_SENT")).alias("theme_open_rate"),
  (f.sum("THEME_CLICK") / f.sum("THEME_SENT")).alias("theme_click_rate"),
  f.sum("DEPT_SENT").alias("DEPT_SENT"),
  f.sum("DEPT_OPEN").alias("DEPT_OPEN"),
  f.sum("DEPT_CLICK").alias("DEPT_CLICK"),
  f.sum("DEPT_UNSUBSCRIBE").alias("DEPT_UNSUBSCRIBE"),
  (f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).alias("dept_open_rate"),
  (f.sum("DEPT_CLICK") / f.sum("DEPT_SENT")).alias("dept_click_rate"),
)

In [0]:
test_health_aggs.display()

In [0]:
test_health_aggs_solo = test_health_aggs.select("health_dim_seg", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()
test_health_aggs_solo.groupBy("health_dim_seg").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

### Qualtity Dim Seg

In [0]:

test_quality_aggs = test_cds.groupBy("quality_dim_seg", "THEME", "DEPT").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("THEME_SENT").alias("THEME_SENT"),
  f.sum("THEME_OPEN").alias("THEME_OPEN"),
  f.sum("THEME_CLICK").alias("THEME_CLICK"),
  f.sum("THEME_UNSUBSCRIBE").alias("THEME_UNSUBSCRIBE"),
  (f.sum("THEME_OPEN") / f.sum("THEME_SENT")).alias("theme_open_rate"),
  (f.sum("THEME_CLICK") / f.sum("THEME_SENT")).alias("theme_click_rate"),
  f.sum("DEPT_SENT").alias("DEPT_SENT"),
  f.sum("DEPT_OPEN").alias("DEPT_OPEN"),
  f.sum("DEPT_CLICK").alias("DEPT_CLICK"),
  f.sum("DEPT_UNSUBSCRIBE").alias("DEPT_UNSUBSCRIBE"),
  (f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).alias("dept_open_rate"),
  (f.sum("DEPT_CLICK") / f.sum("DEPT_SENT")).alias("dept_click_rate"),
)

In [0]:
test_quality_aggs.display()

In [0]:
test_quality_aggs_solo = test_quality_aggs.select("quality_dim_seg", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()

test_quality_aggs_solo.groupBy("quality_dim_seg").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

### Convenience dim seg

In [0]:

test_convenience_aggs = test_cds.groupBy("convenience_dim_seg", "THEME", "DEPT").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("THEME_SENT").alias("THEME_SENT"),
  f.sum("THEME_OPEN").alias("THEME_OPEN"),
  f.sum("THEME_CLICK").alias("THEME_CLICK"),
  f.sum("THEME_UNSUBSCRIBE").alias("THEME_UNSUBSCRIBE"),
  (f.sum("THEME_OPEN") / f.sum("THEME_SENT")).alias("theme_open_rate"),
  (f.sum("THEME_CLICK") / f.sum("THEME_SENT")).alias("theme_click_rate"),
  f.sum("DEPT_SENT").alias("DEPT_SENT"),
  f.sum("DEPT_OPEN").alias("DEPT_OPEN"),
  f.sum("DEPT_CLICK").alias("DEPT_CLICK"),
  f.sum("DEPT_UNSUBSCRIBE").alias("DEPT_UNSUBSCRIBE"),
  (f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).alias("dept_open_rate"),
  (f.sum("DEPT_CLICK") / f.sum("DEPT_SENT")).alias("dept_click_rate"),
)

In [0]:
test_convenience_aggs.display()

In [0]:
test_convenience_aggs_solo = test_convenience_aggs.select("convenience_dim_seg", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()

test_convenience_aggs_solo.groupBy("convenience_dim_seg").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

### Variety Seeking Dim Seg

In [0]:
test_variety_aggs = test_cds.groupBy("variety_seeking_dim_seg","THEME", "DEPT").agg(
  f.countDistinct("guid").alias("Unique_Guids"),
  f.countDistinct("ehhn").alias("Unique_Households"),
  f.sum("THEME_SENT").alias("THEME_SENT"),
  f.sum("THEME_OPEN").alias("THEME_OPEN"),
  f.sum("THEME_CLICK").alias("THEME_CLICK"),
  f.sum("THEME_UNSUBSCRIBE").alias("THEME_UNSUBSCRIBE"),
  (f.sum("THEME_OPEN") / f.sum("THEME_SENT")).alias("theme_open_rate"),
  (f.sum("THEME_CLICK") / f.sum("THEME_SENT")).alias("theme_click_rate"),
  f.sum("DEPT_SENT").alias("DEPT_SENT"),
  f.sum("DEPT_OPEN").alias("DEPT_OPEN"),
  f.sum("DEPT_CLICK").alias("DEPT_CLICK"),
  f.sum("DEPT_UNSUBSCRIBE").alias("DEPT_UNSUBSCRIBE"),
  (f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).alias("dept_open_rate"),
  (f.sum("DEPT_CLICK") / f.sum("DEPT_SENT")).alias("dept_click_rate"),
)

In [0]:
test_variety_aggs.display()

In [0]:
test_variety_aggs_solo = test_variety_aggs.select("variety_seeking_dim_seg", "DEPT", "DEPT_OPEN","DEPT_UNSUBSCRIBE", "DEPT_SENT").dropDuplicates()

test_variety_aggs_solo.groupBy("variety_seeking_dim_seg").agg(f.sum("DEPT_OPEN"), f.sum("DEPT_SENT"), f.sum("DEPT_OPEN") / f.sum("DEPT_SENT")).display()

##ACDS X TEST